In [1]:
#import the libraries
from flask import Flask, render_template, request
import joblib
import numpy as np
from tensorflow.keras.models import load_model

app = Flask(__name__)

In [2]:
# Load trained models
models = {
    "Random Forest": joblib.load("rf_model_retrained.pkl"),
    "Logistic Regression": joblib.load("logistic_model_retrained.pkl"),
    "XGBoost": joblib.load("xgb_model_retrained.pkl"),
    "SVM": joblib.load("svm_model_retrained.pkl"),
    "DNN": load_model("dnn_model_retrained.h5")
}

c:\Users\sowja\anaconda3\Lib\pickle.py:1759: UserWarning: [16:49:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\data\../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


In [3]:
@app.route('/', methods=['GET', 'POST'])
def index():
    result = None
    if request.method == 'POST':
        model_name = request.form['model']
        try:
            # Collect input features
            features = [
                float(request.form['InscClaimAmtReimbursed']),
                float(request.form['DeductibleAmtPaid']),
                int(request.form['ChronicCond_Diabetes']),
                int(request.form['ChronicCond_Heartfailure'])
            ]

            # Convert to numpy array and reshape
            features = np.array([features])

            # Make prediction
            if model_name == "DNN":
                prediction = models[model_name].predict(features, verbose=0)
                predicted_class = 1 if prediction[0][0] > 0.5 else 0
            else:
                predicted_class = models[model_name].predict(features)[0]

            result = "Fraud" if predicted_class == 1 else "Not Fraud"

        except Exception as e:
            result = f"Error: {str(e)}"

    return render_template("index.html", result=result)

In [ ]:
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [03/Jun/2025 17:06:05] "GET / HTTP/1.1" 200 -
